In [14]:
## Imports

In [15]:
import contextlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
%matplotlib inline
import re
import requests
import sqlite3
import bs4
from bs4 import BeautifulSoup
import time
import random

In [16]:
DATABASE = '/Users/pandabear/springboard/CapstoneTwoProject/data/interim/walkscore.db'
HTML_DIRECTORY = '/Users/pandabear/springboard/CapstoneTwoProject/data/interim'
DEBUG = False

In [17]:
# The parsed html listing data is the interim directory
dir_folder = '/Users/pandabear/springboard/CapstoneTwoProject/data/interim/'
listing_df = pd.read_csv(os.path.join(HTML_DIRECTORY, 'listing_df_cleaned.csv'), index_col='listing_id')
print(f'Size of dataframe: {listing_df.shape}')
listing_df.head()

Size of dataframe: (9712, 49)


,listing_title,listing_nh,listing_city,listing_date,listing_price,listing_bedrooms,listing_bathrooms,listing_sqft,listing_address,listing_info,...,housing_duplex,housing_flat,housing_house,housing_inlaw,housing_townhouse,is_consistent_metro,is_rent_controlled,pets_allowed,is_remodelled,has_amenities
listing_id,,,,,,,,,,,,,,,,,,,,,
7520103440,Beautiful and Spacious NEW remodeled Silicon V...,fremont / union city / newark,Fremont,2022-08-11 09:51:58-07:00,3800,3,2,1495.0,Newport Dr,open house dates saturday 2022-08-13 sun...,...,0,0,0,0,0,True,0,0,0,0
7520292010,"Spacious, bright live/work loft with all utili...",oakland west,Oakland,2022-08-11 16:49:29-07:00,2450,1,1,925.0,1468 32nd St near louise,application fee details: $45 cats are OK - p...,...,0,0,0,1,0,True,0,0,0,0
7520905091,Highly desired Vista neighborhood with a charm...,vallejo / benicia,Vallejo,2022-08-13 09:04:22-07:00,2100,2,1,904.0,226 Edgemont Avenue near Edgemont Avenue,EV charging air conditioning cats are OK - ...,...,0,0,1,0,0,True,0,0,0,1
7520907681,lovely 2-bedroom 2 bath home with open floor p...,vallejo / benicia,Vallejo,2022-08-13 09:09:26-07:00,2200,2,2,1420.0,1509 Sacramento Street near Sacramento Street,EV charging air conditioning cats are OK - ...,...,0,0,1,0,0,True,0,0,0,1
7520608095,"$3,935 / 2br - 1266ft2 - NEW LOW RATE! Spaciou...",dublin / pleasanton / livermore,Pleasanton,2022-08-12 12:34:49-07:00,3935,2,2,NaN,1552 E Gate Way,air conditioning application fee details: $5...,...,0,0,0,0,0,True,0,0,0,1


In [18]:
def get_walk_score(conn, full_address):
    query = 'SELECT walk_score, transit_score, bike_score FROM walkscore WHERE full_address = ?'
    cur = conn.cursor()
    cur.execute(query, (full_address,))
    row = cur.fetchone()
    if row:
        return (row[0] or None, row[1] or None, row[2] or None,)


def insert_walk_score(conn, full_address, walk_score, transit_score, bike_score):
    query = 'INSERT INTO walkscore (full_address, walk_score, transit_score, bike_score) VALUES (?, ?, ?, ?)'
    cur = conn.cursor()
    cur.execute(query, (full_address, walk_score, transit_score, bike_score,))
    conn.commit()

In [19]:
# Add walk score if there is an address
walk_score_root_URL = 'https://www.walkscore.com/score/'
# walk score website url address example format '562-29th-st-oakland'

def get_address_score(conn, address, city):
    full_address = str(address) + " " + str(city)
    full_address = full_address.replace('"','')
    
    # Check address is not np.nan
    if address == np.nan:
        walk_score = None
        transit_score = None
        bike_score = None
        return insert_walk_score(conn, full_address, walk_score, transit_score, bike_score)
    
    # If address is valid, check it is in the walkscore database
    if get_walk_score(conn, full_address):
        return get_walk_score(conn, full_address)
    
    else:
        # Else, scrape the scores from the walkscore website
        address_url = "-".join(str(address).split())
        city_url = "-".join(str(city).split())
        walk_score_url = walk_score_root_URL + address_url + "-" + city_url

    #     print(f'URL of walk score to scrape: {walk_score_url}')
        r = requests.get(walk_score_url)
    #     r.raise_for_status()
        if r.status_code != 200:
            print('Request: {}; Status code: {}'.format(r, r.status_code))
            walk_score = None
            transit_score = None
            bike_score = None
            insert_walk_score(conn, full_address, walk_score, transit_score, bike_score)
            return 

        soup = bs4.BeautifulSoup(r.text, 'html.parser')

        # Get walk score
        div_walk = soup.find("div", {"class": "block-header-badge score-info-link", "data-eventsrc": "score page walk badge"})
        if div_walk == None:
            walk_score = None
        else:
            img_walk = div_walk.findChild('img')
            walk_text_before_regex = img_walk.get('src')
            walk_score = int(re.search('/(\d+)\.svg$', walk_text_before_regex)[1])

        # Get transit score
        div_transit = soup.find("div", {"class": "block-header-badge score-info-link", "data-eventsrc": "score page transit badge"})
        if div_transit == None:
            transit_score = None
        else:
            img_transit = div_transit.findChild('img')
            transit_text_before_regex = img_transit.get('src')
            transit_score = int(re.search('/(\d+)\.svg$', transit_text_before_regex)[1])

        # Get biking score
        div_bike = soup.find("div", {"class": "block-header-badge score-info-link", "data-eventsrc": "score page bike badge"})
        if div_bike == None:
            bike_score = None
        else:
            img_bike = div_bike.findChild('img')
            bike_text_before_regex = img_bike.get('src')
            bike_score = int(re.search('/(\d+)\.svg$', bike_text_before_regex)[1])

    # Add new scores to walkscore database
    insert_walk_score(conn, full_address, walk_score, transit_score, bike_score)
    
    # Add a delay between requests
    time.sleep(random.uniform(1.5, 2.5))

    return walk_score[:1] if DEBUG else [walk_score,transit_score,bike_score]

In [20]:
test_df = listing_df

In [21]:
# Add walkscore data if there is an address in the listing
with contextlib.closing(sqlite3.connect(DATABASE)) as conn:
    test_df['address_score'] = test_df[['listing_address', 'listing_city']].apply(lambda x: get_address_score(conn, *x), axis=1)

Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404
Request: <Response [404]>; Status code: 404


In [22]:
test_df.head()

,listing_title,listing_nh,listing_city,listing_date,listing_price,listing_bedrooms,listing_bathrooms,listing_sqft,listing_address,listing_info,...,housing_flat,housing_house,housing_inlaw,housing_townhouse,is_consistent_metro,is_rent_controlled,pets_allowed,is_remodelled,has_amenities,address_score
listing_id,,,,,,,,,,,,,,,,,,,,,
7520103440,Beautiful and Spacious NEW remodeled Silicon V...,fremont / union city / newark,Fremont,2022-08-11 09:51:58-07:00,3800,3,2,1495.0,Newport Dr,open house dates saturday 2022-08-13 sun...,...,0,0,0,0,True,0,0,0,0,"(52, 29, 56)"
7520292010,"Spacious, bright live/work loft with all utili...",oakland west,Oakland,2022-08-11 16:49:29-07:00,2450,1,1,925.0,1468 32nd St near louise,application fee details: $45 cats are OK - p...,...,0,0,1,0,True,0,0,0,0,"(80, 57, 98)"
7520905091,Highly desired Vista neighborhood with a charm...,vallejo / benicia,Vallejo,2022-08-13 09:04:22-07:00,2100,2,1,904.0,226 Edgemont Avenue near Edgemont Avenue,EV charging air conditioning cats are OK - ...,...,0,1,0,0,True,0,0,0,1,"(61, 31, 44)"
7520907681,lovely 2-bedroom 2 bath home with open floor p...,vallejo / benicia,Vallejo,2022-08-13 09:09:26-07:00,2200,2,2,1420.0,1509 Sacramento Street near Sacramento Street,EV charging air conditioning cats are OK - ...,...,0,1,0,0,True,0,0,0,1,"(74, None, None)"
7520608095,"$3,935 / 2br - 1266ft2 - NEW LOW RATE! Spaciou...",dublin / pleasanton / livermore,Pleasanton,2022-08-12 12:34:49-07:00,3935,2,2,NaN,1552 E Gate Way,air conditioning application fee details: $5...,...,0,0,0,0,True,0,0,0,1,"(53, 23, 72)"


In [23]:
# Separate scores into their own columns
test_df['walk_score'] = test_df['address_score'].apply(lambda x: x[0] if x else np.nan)
test_df['transit_score'] = test_df['address_score'].apply(lambda x: x[1] if x else np.nan)
test_df['bike_score'] = test_df['address_score'].apply(lambda x: x[2] if x else np.nan)

In [24]:
no_listing_address = len(test_df['listing_address'].notnull())
missing_walkscore = test_df['walk_score'].isnull().sum()
missing_transitscore = test_df['transit_score'].isnull().sum()
missing_bikescore = test_df['bike_score'].isnull().sum()

print(f"Number of listing addresses: {no_listing_address}")
print(f"Number of missing walk scores: {missing_walkscore}, {missing_walkscore*100/no_listing_address:.1f}% of addresses")
print(f"Number of missing transit scores: {missing_transitscore}, {missing_transitscore*100/no_listing_address:.1f}% of addresses")
print(f"Number of missing bike scores: {missing_bikescore}, {missing_bikescore*100/no_listing_address:.1f}% of addresses")

Number of listing addresses: 9712
Number of missing walk scores: 302, 3.1% of addresses
Number of missing transit scores: 1747, 18.0% of addresses
Number of missing bike scores: 1223, 12.6% of addresses


In [25]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime

In [26]:
# Convert listing_nh into one hot encoded columns
df = pd.get_dummies(listing_df, columns=['listing_nh','listing_bathrooms','listing_bedrooms'])

In [27]:
df.head()

,listing_title,listing_city,listing_date,listing_price,listing_sqft,listing_address,listing_info,listing_body,listing_url,listing_first_image,...,listing_bathrooms_3.5,listing_bathrooms_4+,listing_bathrooms_shared,listing_bedrooms_0,listing_bedrooms_1,listing_bedrooms_2,listing_bedrooms_3,listing_bedrooms_4,listing_bedrooms_5,listing_bedrooms_6+
listing_id,,,,,,,,,,,,,,,,,,,,,
7520103440,Beautiful and Spacious NEW remodeled Silicon V...,Fremont,2022-08-11 09:51:58-07:00,3800,1495.0,Newport Dr,open house dates saturday 2022-08-13 sun...,Great location 3 Bed 2 Bath Single Family Hous...,https://sfbay.craigslist.org/eby/apa/d/fremont...,https://images.craigslist.org/00C0C_iEcKsVmzdg...,...,0,0,0,0,0,0,1,0,0,0
7520292010,"Spacious, bright live/work loft with all utili...",Oakland,2022-08-11 16:49:29-07:00,2450,925.0,1468 32nd St near louise,application fee details: $45 cats are OK - p...,"Spacious, bright live/work loft with all utili...",https://sfbay.craigslist.org/eby/apa/d/emeryvi...,https://images.craigslist.org/00n0n_9bLXcwFIi1...,...,0,0,0,0,1,0,0,0,0,0
7520905091,Highly desired Vista neighborhood with a charm...,Vallejo,2022-08-13 09:04:22-07:00,2100,904.0,226 Edgemont Avenue near Edgemont Avenue,EV charging air conditioning cats are OK - ...,Highly desired Vista neighborhood with a charm...,https://sfbay.craigslist.org/eby/apa/d/vallejo...,https://images.craigslist.org/00c0c_8OXB1Zm4PN...,...,0,0,0,0,0,1,0,0,0,0
7520907681,lovely 2-bedroom 2 bath home with open floor p...,Vallejo,2022-08-13 09:09:26-07:00,2200,1420.0,1509 Sacramento Street near Sacramento Street,EV charging air conditioning cats are OK - ...,lovely 2-bedroom 2 bath home with open floor p...,https://sfbay.craigslist.org/eby/apa/d/vallejo...,https://images.craigslist.org/00303_28XoaVJ8dA...,...,0,0,0,0,0,1,0,0,0,0
7520608095,"$3,935 / 2br - 1266ft2 - NEW LOW RATE! Spaciou...",Pleasanton,2022-08-12 12:34:49-07:00,3935,NaN,1552 E Gate Way,air conditioning application fee details: $5...,Available Date: Now Apartment #: 110 Size: 1...,https://sfbay.craigslist.org/eby/apa/d/pleasan...,https://images.craigslist.org/00g0g_ikx5DdQu5V...,...,0,0,0,0,0,1,0,0,0,0


In [28]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='listing_price'), 
                                                    listing_df['listing_price'], test_size=0.3, 
                                                    random_state=1)


In [29]:
X_train.shape, X_test.shape

((6798, 146), (2914, 146))

In [30]:
y_train.shape, y_test.shape

((6798,), (2914,))

In [31]:
#Baseline listing price prediction: mean of the listing data
train_mean = y_train.mean()
print(f'Training mean: {train_mean}')

dumb_reg = DummyRegressor(strategy='mean')
dumb_reg.fit(X_train, y_train)
y_te_pred = train_mean * np.ones(len(y_test))

def mae(y, ypred):
    """Mean absolute error.
    Calculate the mean absolute error of the arguments
    Arguments:
    y -- the observed values
    ypred -- the predicted values
    """
    abs_error = np.abs(y - ypred)
    mae = np.mean(abs_error)
    return mae
print(f'MAE training: {mae(y_train, y_tr_pred)}')
print(f'MAE test: {mae(y_test, y_te_pred)}')


Training mean: 3356.339511621065


NameError: name 'y_tr_pred' is not defined

In [ ]:
# Save 'listing_title', 'listing_date', 'listing_info', 'listing_info_split', 'listing_body', 'listing_address', 'listing_city','listing_url', 'listing_first_image' columns from the train/test data into names_train and names_test
# Drop inplace those columns from X_train and X_test
names_list = ['listing_title', 'listing_date', 'listing_info', 'listing_info_split','listing_body','listing_address','listing_city','listing_url','listing_first_image']
names_train = X_train[names_list]
names_test = X_test[names_list]

In [ ]:
X_train.drop(columns=names_list, inplace=True)
X_test.drop(columns=names_list, inplace=True)
X_train.shape, X_test.shape

In [ ]:
y_train.head()

In [ ]:
X_train.dtypes

In [ ]:
df.listing_sqft.describe()

In [ ]:
print(f'The column listing_sqft is missing {df.listing_sqft.isnull().sum()} values')

In [ ]:
# Impute the missing sqft with the mean 


## Linear Regression Model

In [ ]:
pipe = make_pipeline(
    SimpleImputer(strategy='median'), 
    StandardScaler(),
    SelectKBest(f_regression, k=15),
    LinearRegression()
)
pipe.fit(X_train, y_train)

In [ ]:
y_tr_pred = pipe.predict(X_train)
y_te_pred = pipe.predict(X_test)

In [ ]:
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

In [ ]:
mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)